In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
import numpy as np 
import requests
import matplotlib.pyplot as plt
import os

In [ ]:
initial_url = "https://papers.nips.cc"
url_complement = "/paper_files/paper/"

complet_url = initial_url + url_complement

years = list(range(1987, 2024))

neurips_papers = {
    "Year" : [], 
    "Title" : [], 
    "Authors" : [],
    "Number of authors" : [],
    "Abstract" : [],
    "Citations" : [],
    "Paper file" : []
}

initial_url = "https://papers.nips.cc"
url_complement = "/paper_files/paper/"

folder_to_save = "./NeurIPS"

if not os.path.isdir(folder_to_save) : 
    os.makedirs(folder_to_save)

for year in years : 
    conference_url = initial_url + url_complement + str(year)
    content = requests.get(conference_url).text
    scraper = BeautifulSoup(content, parser="lxml")
    papers = scraper.find_all("li", class_="conference")
    if len(papers) == 0 : 
        papers = scraper.find_all("li", class_="none")
    i = 0
    for paper in papers: 
        paper_tile = paper.a.text
        paper_authors = paper.i.text
        paper_link = paper.a.get("href")
        
        paper_link = initial_url + paper_link

        content = requests.get(paper_link).text
        scraper = BeautifulSoup(content, parser="lxml")

        abstract = scraper.find_all("p")[3].text
        if scraper.find("a", class_="btn btn-primary btn-spacer") is None : 
            pdf_link = initial_url + scraper.find_all("a", class_="btn btn-light btn-spacer")[-1].get("href")
        else : 
            pdf_link = initial_url + scraper.find("a", class_="btn btn-primary btn-spacer").get("href")

        neurips_papers["Year"].append(year)
        neurips_papers["Title"].append(paper_tile)
        neurips_papers["Authors"].append(paper_authors)
        neurips_papers["Number of authors"].append(len(paper_authors.split(",")))
        neurips_papers["Abstract"].append(abstract)
        neurips_papers["Citations"].append(0)
        neurips_papers["Paper file"].append(pdf_link)
        
        print(f"Year : {year}, paper {i+1}/{len(papers)}")
        i += 1

    neurips_year_df = pd.DataFrame(neurips_papers)
    # sauvegarder chaque année
    neurips_year_df.to_csv(os.path.join(folder_to_save, "NeurIPS_papers.csv"), index=False)